<a href="https://colab.research.google.com/github/APYazdi/Crypto-Sentiment-Analysis/blob/main/FinBERT_Crypto_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Getting Dependecies**


In [ ]:
!pip install transformers

In [ ]:
!pip install pygooglenews --upgrade

In [ ]:
!pip install wandb

In [5]:
import torch
import pandas as pd
import wandb 


In [6]:
from pygooglenews import GoogleNews
gn = GoogleNews()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [7]:
# search for the best matching articles that mention LUNA and 
# do not mention BTC (over the past 6 month)
#search = gn.search('LUNA -BTC', when = '6m')


In [8]:
#List of cryptos to capture headlines for 
#fetch headlines from list of cryptos

headline_list = []
crypto_list = []

cryptos = ['bitcoin', 'ethereum','fantom','luna','avalanche','solana']
for crypto in cryptos:
  response = gn.search(crypto)
  for r in response['entries']:
    headline_list.append(r['title'])
    crypto_list.append(crypto)

    

In [9]:
#Setting up project in wandb to push results into a wandb Table for easy visualization & analysis
#MUST HAVE SIGNED IN "https://wandb.ai/home" for API access code


wandb.init(project='Crypto Sentiment Analysis')
headline_table = wandb.Table(columns=["Headline","Crypto", "Postive", "Negative","Neutral"])

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#Peek at headlines captured 

for h in headline_list:
  print(h)

In [ ]:
#Loop through crypto list capturing sentiment prediction into a 3 column matrix for Positive, Negative, Neutral  
def chunkList(list,n):
  for i in range(0, len(list),n):
    yield list[i:i+n]

n = 0
chunkSize = 100
model.eval()
for lines, cryptos in zip(chunkList(headline_list,chunkSize), chunkList(crypto_list,chunkSize)):
  input = tokenizer(lines,padding=True, truncation=True, return_tensors='pt')
  outputs = model(**input)
  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
  print(f"{n+1}/{int(len(headline_list)/chunkSize)}")
  for headline,crypto,pos,neg,neutr in zip(lines, cryptos, prediction[:,0].tolist(),prediction[:,1].tolist(),prediction[:,2].tolist()):
    headline_table.add_data(headline,crypto,pos,neg,neutr)
  n = n + 1

In [ ]:
#Push model predictions into a wandb table

wandb.run.log({"Crypto Sentiment Analysis Table": headline_table})
wandb.run.finish()